In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# !mkdir input_data
# !unzip "/content/gdrive/My Drive/Bases_VisagioHackathon.zip" -d "/content/input_data"


Mounted at /content/gdrive
Archive:  /content/gdrive/My Drive/Bases_VisagioHackathon.zip
  inflating: /content/input_data/Base_tweets_saúde_Aos_Fatos.xlsx  
   creating: /content/input_data/FakeHealthData/
  inflating: /content/input_data/FakeHealthData/Base News Releases.csv  
  inflating: /content/input_data/FakeHealthData/Base Story.csv  
  inflating: /content/input_data/FakeHealthData/de_para_news_questions.csv  
  inflating: /content/input_data/FakeHealthData/de_para_story_questions.csv  
  inflating: /content/input_data/FakeHealthData/news_tweets.csv  
  inflating: /content/input_data/FakeHealthData/story_tweets.csv  


In [ ]:
# !git clone https://github.com/huggingface/transformers
# %cd transformers
# !pip install .
# !git pull
# !pip install --upgrade .
# !pip install sentencepiece
# !pip install transformers
# import torch
# import sentencepiece
# import gc
# gc.collect()

In [ ]:
import pandas as pd
base_news_releases = pd.read_csv('/content/input_data/FakeHealthData/Base News Releases.csv',encoding='utf-8',sep=';')
base_news_releases.head(2)

,Unnamed: 0,url,text,images,top_img,keywords,authors,canonical_link,title_1,meta_data,movies,publish_date,source,summary,id,link,title_2,description,original_title,rating,reviewers,category,tags,source_link,news_id,news_source,Our Review Summary,Why This Matters,img,question_0_answer,question_0_explanation,question_1_answer,question_1_explanation,question_2_answer,question_2_explanation,question_3_answer,question_3_explanation,question_4_answer,question_4_explanation,question_5_answer,question_5_explanation,question_6_answer,question_6_explanation,question_7_answer,question_7_explanation,question_8_answer,question_8_explanation,question_9_answer,question_9_explanation
0,0,https://web.archive.org/web/20181218015531/htt...,"MADISON, Wis. -- More than 700 million adults ...",['https://web.archive.org/web/20181218015531im...,https://web.archive.org/web/20181218015531im_/...,[],[],https://web.archive.org/web/20181218015531/htt...,Tiny implantable device short-circuits hunger ...,"{'viewport': 'width=device-width, initial-scal...",[],2018-12-29 02:00:00,https://web.archive.org,NaN,news_reviews_00000,https://www.healthnewsreview.org/news-release-...,Implantable device ‘aids weight loss’? Let’s w...,This news release speculates that a study in r...,Tiny implantable device short-circuits hunger ...,2,"['Andrew Holtz, MPH', 'Yoni Freedhoff, MD', 'K...",University news release,"['implant', 'University of Wisconsin-Madison',...",https://www.eurekalert.org/pub_releases/2018-1...,news_reviews_00000,University of Wisconsin-Madison,UW-Madison graduate student Guang Yao (left) a...,The history of medical research is littered wi...,NaN,Not Satisfactory,Although this device is still in early testing...,Satisfactory,The release states that the device is experime...,Satisfactory,The release does report that there is an exist...,Not Satisfactory,"The release does not discuss potential harms, ...",Satisfactory,We will give the release a satisfactory rating...,Not Satisfactory,"By definition, researchers do not know whether...",Not Satisfactory,The lead sentence of the release proclaims tha...,Not Satisfactory,The news release refers to “several advantages...,Satisfactory,The release states that funding was provided b...,Not Satisfactory,The release says this device could be an impro...
1,1,https://web.archive.org/web/20181217203805/htt...,"Wilmington, DE, December 17, 2018 - The CRISPR...",['https://web.archive.org/web/20181217203805im...,https://web.archive.org/web/20181217203805im_/...,[],[],https://web.archive.org/web/20181217203805/htt...,Scientists report CRISPR restores effectivenes...,"{'viewport': 'width=device-width, initial-scal...",[],2018-12-29 02:00:00,https://web.archive.org,NaN,news_reviews_00001,https://www.healthnewsreview.org/news-release-...,Summary of CRISPR lung cancer research downpla...,This release needed a strong caution that obse...,Scientists report CRISPR restores effectivenes...,3,"['Mary Chris Jaklevic', 'Andrae Vandross, MD',...",Academic medical center news release,"['Christiana Care Health System', 'CRISPR', 'l...",https://www.eurekalert.org/pub_releases/2018-1...,news_reviews_00001,Christiana Care Health System,This news release reports on preliminary resea...,Chemotherapy is a mainstay of treatment for ad...,https://www.healthnewsreview.org/wp-content/up...,Not Satisfactory,The news release mentioned this as a potential...,Satisfactory,The news release makes it clear that this is n...,Satisfactory,The news release mentions “many efforts under ...,Not Satisfactory,The news release doesn’t talk about specific h...,Not Satisfactory,In the second paragraph the news release expla...,Not Satisfactory,The news release uses cautious language. Howev...,Satisfactory,There’s no disease-mongering. The release says...,Not Satisfactory,The release is unable to claim a benefit in hu...,Satisfactory,The funders (NIH and State of Delaware) are no...,Satisfactory,The lead researcher calls the study “an exciti...


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

In [ ]:
def summary_generate(text,model,tokenizer):
  '''
    Generates the summary of a text based on a given model.
  '''

  if len(text.split())>4200:
    max_length_in = 600
    min_length_in = 300
  
  elif ((len(text.split())<4200) and (len(text.split())>3000)):
    max_length_in = 500
    min_length_in = 200

  else:
    max_length_in = 200
    min_length_in = 80
  
  batch        = tokenizer.prepare_seq2seq_batch([text], truncation=True, padding='longest', return_tensors="pt").to(torch_device)
  translated   = model.generate(
                                **batch,
                                max_length = 500, # max length of summary
                                min_length = 200, # min length of summary
                                do_sample = True, 
                                temperature = 3.0,
                                top_k =30,
                                top_p=0.70,
                                repetition_penalty = 1.2,
                                length_penalty = 5, # if more than 1 encourage model to generate #larger sequences
                                num_return_sequences=1) # no of summary you want to generate)
  tgt_text     = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text[0]


In [ ]:
model_name   = 'google/pegasus-reddit_tifu'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer    = PegasusTokenizer.from_pretrained(model_name)
model        = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
base_news_releases['text'] = base_news_releases['text'].astype(str)

In [ ]:
summary_result = []
for i in range(base_news_releases.shape[0]):
  try:
  
    text = base_news_releases.loc[i,'text']
    summary = summary_generate(text,model,tokenizer)
    summary_result.append(summary)
    print('Summary Generated:-{}'.format(summary))
    print('\n')
  except:
    pass

Summary Generated:-zapping the stomach to reduce appetite could be as simple as the stomach's churning motion, which stimulates a nerve that controls the brain, according to the first device to be published in a peer-reviewed journal with a full-grown human author. The researchers say it could also be used to control appetite in patients who are obese, or who suffer from conditions such as metabolic syndrome that inhibit their appetite. "Obesity is an epidemic worldwide that is on the rise, and our team thinks this is a promising technique to treat it," said senior author of the study and a post-doctoral researcher at the University of South Carolina. "We believe our device may be an important step forward for weight loss management and could be an important treatment for obesity-related disorders including heart disease and type 2 diabetes," said co-lead author of the paper and a doctoral student at the same time. "We are still in the early design phase, and there is potential for mor

In [ ]:
!pip install rouge_score

In [ ]:
# num = 56
# t1 = summary_result[num]
# t2 = base_news_releases.loc[num,'Our Review Summary']
# text = base_news_releases.loc[num,'text']

In [ ]:
from rouge_score import rouge_scorer



In [ ]:
base_news_releases.head()

,Unnamed: 0,url,text,images,top_img,keywords,authors,canonical_link,title_1,meta_data,movies,publish_date,source,summary,id,link,title_2,description,original_title,rating,reviewers,category,tags,source_link,news_id,news_source,Our Review Summary,Why This Matters,img,question_0_answer,question_0_explanation,question_1_answer,question_1_explanation,question_2_answer,question_2_explanation,question_3_answer,question_3_explanation,question_4_answer,question_4_explanation,question_5_answer,question_5_explanation,question_6_answer,question_6_explanation,question_7_answer,question_7_explanation,question_8_answer,question_8_explanation,question_9_answer,question_9_explanation
0,0,https://web.archive.org/web/20181218015531/htt...,"MADISON, Wis. -- More than 700 million adults ...",['https://web.archive.org/web/20181218015531im...,https://web.archive.org/web/20181218015531im_/...,[],[],https://web.archive.org/web/20181218015531/htt...,Tiny implantable device short-circuits hunger ...,"{'viewport': 'width=device-width, initial-scal...",[],2018-12-29 02:00:00,https://web.archive.org,NaN,news_reviews_00000,https://www.healthnewsreview.org/news-release-...,Implantable device ‘aids weight loss’? Let’s w...,This news release speculates that a study in r...,Tiny implantable device short-circuits hunger ...,2,"['Andrew Holtz, MPH', 'Yoni Freedhoff, MD', 'K...",University news release,"['implant', 'University of Wisconsin-Madison',...",https://www.eurekalert.org/pub_releases/2018-1...,news_reviews_00000,University of Wisconsin-Madison,UW-Madison graduate student Guang Yao (left) a...,The history of medical research is littered wi...,NaN,Not Satisfactory,Although this device is still in early testing...,Satisfactory,The release states that the device is experime...,Satisfactory,The release does report that there is an exist...,Not Satisfactory,"The release does not discuss potential harms, ...",Satisfactory,We will give the release a satisfactory rating...,Not Satisfactory,"By definition, researchers do not know whether...",Not Satisfactory,The lead sentence of the release proclaims tha...,Not Satisfactory,The news release refers to “several advantages...,Satisfactory,The release states that funding was provided b...,Not Satisfactory,The release says this device could be an impro...
1,1,https://web.archive.org/web/20181217203805/htt...,"Wilmington, DE, December 17, 2018 - The CRISPR...",['https://web.archive.org/web/20181217203805im...,https://web.archive.org/web/20181217203805im_/...,[],[],https://web.archive.org/web/20181217203805/htt...,Scientists report CRISPR restores effectivenes...,"{'viewport': 'width=device-width, initial-scal...",[],2018-12-29 02:00:00,https://web.archive.org,NaN,news_reviews_00001,https://www.healthnewsreview.org/news-release-...,Summary of CRISPR lung cancer research downpla...,This release needed a strong caution that obse...,Scientists report CRISPR restores effectivenes...,3,"['Mary Chris Jaklevic', 'Andrae Vandross, MD',...",Academic medical center news release,"['Christiana Care Health System', 'CRISPR', 'l...",https://www.eurekalert.org/pub_releases/2018-1...,news_reviews_00001,Christiana Care Health System,This news release reports on preliminary resea...,Chemotherapy is a mainstay of treatment for ad...,https://www.healthnewsreview.org/wp-content/up...,Not Satisfactory,The news release mentioned this as a potential...,Satisfactory,The news release makes it clear that this is n...,Satisfactory,The news release mentions “many efforts under ...,Not Satisfactory,The news release doesn’t talk about specific h...,Not Satisfactory,In the second paragraph the news release expla...,Not Satisfactory,The news release uses cautious language. Howev...,Satisfactory,There’s no disease-mongering. The release says...,Not Satisfactory,The release is unable to claim a benefit in hu...,Satisfactory,The funders (NIH and State of Delaware) are no...,Satisfactory,The lead researcher calls the study “an exciti...
2,2,https://web.arch

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=False)


output = []
for i in range(len(summary_result)):
  t1 = summary_result[i]
  t2 = base_news_releases.loc[i,'Our Review Summary']
  text = base_news_releases.loc[i,'text']
  rating = base_news_releases.loc[i,'rating']
  id     = base_news_releases.loc[i,'id']
  
  scores_orignal_summary   = scorer.score(t2,text)
  fmeasure_orig_1          = scores_orignal_summary['rouge1'][2]
  fmeasure_orig_L          = scores_orignal_summary['rougeL'][2]
  
  scores_generated_summary = scorer.score(t1,text)
  fmeasure_gen_1           = scores_generated_summary['rouge1'][2]
  fmeasure_gen_L           = scores_generated_summary['rougeL'][2]
  output.append([id,text,t1,t2,fmeasure_gen_1,fmeasure_gen_L,fmeasure_orig_1,fmeasure_orig_L,rating])

In [ ]:
df = pd.DataFrame(output,columns=['ID','Text','generated_summary','Original_summary',
                           'Rouge_score_Uni_gen','Rouge_score_fulllength_gen',
                           'Rouge_score_Uni_orig','Rouge_score_fulllength_orig','rating'
                           ])

In [ ]:
df['ID'] =list(range(1,301))
df.to_csv('/content/output1.csv',index=False)

In [ ]:
df.head()

,ID,Text,generated_summary,Original_summary,Rouge_score_Uni_gen,Rouge_score_fulllength_gen,Rouge_score_Uni_orig,Rouge_score_fulllength_orig,rating
0,news_reviews_00000,"MADISON, Wis. -- More than 700 million adults ...",zapping the stomach to reduce appetite could b...,UW-Madison graduate student Guang Yao (left) a...,0.328767,0.124533,0.234637,0.128492,2
1,news_reviews_00001,"Wilmington, DE, December 17, 2018 - The CRISPR...",gene-editing systems are already used in cance...,This news release reports on preliminary resea...,0.214180,0.109306,0.097600,0.070400,3
2,news_reviews_00002,About 3 million people in the US are diagnosed...,probiotic supplementation may decrease intesti...,This news release focuses on a study of the gu...,0.336336,0.162162,0.195212,0.099448,1
3,news_reviews_00003,Newswise — Thousands of people can take heart ...,;Australians who are at risk of heart and card...,This news release summarizes findings from a r...,0.319728,0.131519,0.137580,0.089172,1
4,news_reviews_00004,Researchers at Johns Hopkins Medicine have ide...,"""tracers that light up the tau tangles may be ...",The release focuses on two papers that discuss...,0.256911,0.126829,0.154839,0.092166,3


In [ ]:
df.rating.value_counts()

2    109
3    104
1     50
4     32
0      5
Name: rating, dtype: int64